Test for using actionable-recourse, provided on https://github.com/ustunb/actionable-recourse

In order to compare recourse for several similar classifiers, we use cross validation to fit several logistic regression models (Is this the right way?). In the next step, we want to check whether the flipsets generated for one of them apply also for the other classifiers.

In [1]:
import copy
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold as CVGenerator
from sklearn.model_selection import GridSearchCV
import recourse as rs
from recourse.builder import ActionSet #FIX
from recourse.flipset import Flipset #FIX
from recourse.auditor import RecourseAuditor #FIX

import data

In [7]:
url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
df = pd.read_csv(url, skipinitialspace=True)
n_data = len(df) # 30.000
cut_factor = 0.1 # 1 -> full dataset considered (but randomly shuffled)
idxs = np.random.choice(n_data, int((1-cut_factor)*n_data), replace = False)
# randomly choose cut_factor * n samples to fasten analysis
df = df.drop(idxs)
y, X = df.iloc[:, 0], df.iloc[:, 1:]
print(len(df))

3000


NEW: Use Cross validation to train several different classifiers (takes some time!)

In [8]:
alter_C = True
n_splits = 20
if not alter_C:
    clf = LogisticRegression(max_iter=10000)
    cv = cross_validate(clf, X, y, cv=n_splits, return_estimator=True)
    cv_scores = cv['test_score']
    classifiers = np.array(cv['estimator'])

Alternative: Use GridSearchCV on parameter C (takes some time!)

In [18]:
if alter_C:
    cv_generator = CVGenerator(n_splits = 10)
    clf = LogisticRegression(max_iter=10000, solver='lbfgs')

    # this code is for general purpose train/test evaluation using GridSearchCV
    gridsearch = GridSearchCV(
        clf, 
        param_grid={'C':np.logspace(-4, 3, num=n_splits)},
        scoring='accuracy',
        cv=cv_generator,
        verbose=1,
        n_jobs=-1
    )

    gridsearch.fit(X,y)
    grid_search_df = pd.DataFrame(gridsearch.cv_results_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.1min finished


In [19]:
if alter_C:
    # cache a model for each parameter combination, trained on all data
    classifiers = []
    classifier_Cs = []
    for idx, p in tqdm(list(grid_search_df.params.iteritems())):
        model = copy.deepcopy(clf.set_params(**p)).fit(X,y)
        classifiers.append(model)
        classifier_Cs.append(p.items())
    cv_scores = grid_search_df['mean_test_score']
    classifiers = np.array(classifiers)
    # Is it inconsistent to consider score before fitting the whole train set? 
    # But otherwise, it would violate train-test splitting
    # CV + fit is a bit of an overkill here?

NEW: Select those classifiers that achieve performance within certain tolerance

In [26]:
#X_test = X[:3]
#for i, est in enumerate(scores['estimator']):
#    print(scores['test_score'][i], est.predict(X_test))
tolerance = 1*np.std(cv_scores)
good_classifiers = classifiers[cv_scores >= np.max(cv_scores) - tolerance]

print(np.max(cv_scores))
print(cv_scores.std())
print(len(good_classifiers))

0.8006666666666666
0.005204024533476895
15


get predictions

In [21]:
yhat = [clf.predict(X) for clf in good_classifiers]

customize the set of actions and align

In [22]:
action_sets=[]
for clf in good_classifiers:
    ## matrix of features. ActionSet will learn default bounds and step-size.
    A = ActionSet(X)
    ## specify immutable variables
    A['Married'].mutable = False 
    ## can only specify properties for multiple variables using a list
    A[['Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60']].mutable = False 
    A['EducationLevel'].step_direction = 1  ## force conditional immutability.
    A['EducationLevel'].step_size = 1  ## set step-size to a custom value.
    A['EducationLevel'].step_type = "absolute"  ## force conditional immutability.
    A['EducationLevel'].bounds = (0, 3)
    A['TotalMonthsOverdue'].step_size = 1  ## set step-size to a custom value.
    A['TotalMonthsOverdue'].step_type = "absolute"  ## discretize on absolute values of feature rather than percentile values
    A['TotalMonthsOverdue'].bounds = (0, 100)  ## set bounds to a custom value.
    
    ## tells `ActionSet` which directions each feature should move in to produce positive change.
    A.align(clf)
    action_sets.append(A)

NEW: change inputs according to flipsets (takes some time!)

In [27]:
%%capture
j_clf = 0 # flipset generated for j_th classifier TODO later: iterate j_clf?
k_fs = 0  # k-th flipset is applied; MAYDO later: iterate k_fs?
        # when iterating j_clf, we would probably not filter X here...
xs = copy.deepcopy(X.iloc[np.flatnonzero(yhat[j_clf] <= 0)]).to_numpy()
for i in range(len(xs)):
    fs = Flipset(x = xs[i], action_set = action_sets[j_clf], clf = good_classifiers[j_clf])
    fs.populate(enumeration_type = 'distinct_subsets', total_items = 10)
    for j, fi in enumerate(fs._df['feature_idx'][k_fs]):
        xs[i,fi] = fs._df['x_new'][k_fs][j]

NEW: Measure for how many individuals the adjusted input leads to desirable outcomes for each classifier

In [74]:
flips = []
preds = []
for clf in good_classifiers:
    ys = clf.predict(xs)
    preds.append(ys)
    flips.append(np.mean(ys))
flips

[1.0,
 0.16080402010050251,
 0.09045226130653267,
 0.08040201005025126,
 0.17587939698492464,
 0.20100502512562815,
 0.23618090452261306,
 0.20603015075376885,
 0.2914572864321608,
 0.20603015075376885,
 0.271356783919598,
 0.2663316582914573,
 0.2562814070351759,
 0.271356783919598,
 0.2964824120603015]

**TODO:** Use only negatively predicted datapoints; Make whole thing more sound

In [127]:
%%capture
X_new = np.array([copy.deepcopy(X).to_numpy() for clf in good_classifiers])
# X_new[clf that generated flipset; datapoint; feature]
k_fs = 0 # k-th flipset is applied; MAYDO later: iterate k_fs?
for j_clf, clf in enumerate(good_classifiers):
    print("Apply flipsets of classifier", j_clf)
    for i in range(X_new.shape[1]):
        fs = Flipset(x = X_new[j_clf, i, :], action_set = action_sets[j_clf], clf = clf)
        fs.populate(enumeration_type = 'distinct_subsets', total_items = 10)
        X_new[j_clf, i, fs._df['feature_idx'][k_fs]] = fs._df['x_new'][k_fs]

In [128]:
print(xs[10:15,:])
print("----")
print(X_new[0, yhat[0]<=0,:][10:15,:])
print("----")
print(X.to_numpy()[yhat[0]<=0,:][10:15,:])
print("----")
print(preds[0][10:20])
print("----")
print(yhat_new[0, 0, yhat[0]<=0][10:20])
# when applying flipsets, data is only changed for the first 12 datapoints (at least for classifier 0 generating the flipsets)

[[   0    1    0    1    0    0    3 1880   90    0    0    0 1730   90
     1    9    1]
 [   0    1    1    0    0    0    3  678   60    0    0    4  470   42
     1    8    1]
 [   1    0    0    0    1    0    2 5990  430    0    0    0 5370  240
     1   10    1]
 [   0    1    0    0    1    0    2 1590   40    0    3    3 1590 1722
     1   15    1]
 [   0    1    0    1    0    0    2 1880  610    0    0    6 1260    0
     1    7    1]]
----
[[   0    1    0    1    0    0    3 1880   90    0    0    0 1730   90
     1    9    1]
 [   0    1    1    0    0    0    3  678   60    0    0    4  470   42
     1    8    1]
 [   1    0    0    0    1    0    2 5990  430    0    0    0 5370  240
     1   10    1]
 [   0    1    0    0    1    0    2 1590   40    0    3    3 1590 1722
     1   15    1]
 [   0    1    0    1    0    0    2 1880  610    0    0    6 1260    0
     1    7    1]]
----
[[   0    1    0    1    0    0    3 1880   90    0    0    0 1730   90
     1   12    1

In [129]:
yhat_new = np.array([[clf.predict(X_new[j]) for j in range(len(good_classifiers))] for clf in good_classifiers])
print(yhat_new.shape)
# yhat_new[clf that predicted outcome; clf that generated flipset; datapoint]

(15, 15, 3000)


In [130]:
print(X.shape)
print(np.asarray(yhat).shape)
print(X_new.shape)
print(yhat_new.shape)
changed = np.asarray(X) != np.asarray(X_new)
print(changed.mean(axis = 2))

(3000, 17)
(15, 3000)
(15, 3000, 17)
(15, 15, 3000)
[[0.05882353 0.05882353 0.05882353 ... 0.05882353 0.05882353 0.05882353]
 [0.05882353 0.05882353 0.05882353 ... 0.05882353 0.05882353 0.05882353]
 [0.05882353 0.05882353 0.05882353 ... 0.05882353 0.05882353 0.05882353]
 ...
 [0.05882353 0.05882353 0.05882353 ... 0.05882353 0.05882353 0.05882353]
 [0.05882353 0.11764706 0.05882353 ... 0.05882353 0.05882353 0.05882353]
 [0.05882353 0.05882353 0.05882353 ... 0.05882353 0.11764706 0.05882353]]


In [132]:
for j_clf, clf in enumerate(good_classifiers):
    yhat_unpleased = yhat_new[j_clf,:,yhat[j_clf] == 0]
    print(yhat_unpleased.shape) #[datapoint; clf that predicted outcome] why switched?
    flip_accuracy = yhat_unpleased.mean(axis = 0)
    print(flip_accuracy)
    # only the first 17 datapoints are affected WTF -> dimensions have to be messed up somewhere!!...
    print(yhat_unpleased[j_clf].mean())
#flip_accuracy = yhat_new[j_measure, j_generate, np.flatnonzero(yhat[j_generate] == 0)].mean()
#flip_accuracy = yhat_new[:,:,yhat[0] == 0].mean(axis = 2)
#print(flip_accuracy)

(199, 15)
[1.         1.         1.         1.         0.93467337 0.88442211
 0.74874372 0.78894472 0.67839196 0.75376884 0.70351759 0.73869347
 0.74371859 0.72864322 0.72864322]
1.0
(221, 15)
[0.14932127 1.         0.99547511 0.8959276  0.77828054 0.74208145
 0.66515837 0.70588235 0.63348416 0.72850679 0.65158371 0.68325792
 0.68325792 0.67420814 0.6561086 ]
0.3333333333333333
(240, 15)
[0.075      0.32083333 1.         0.85833333 0.73333333 0.72083333
 0.6        0.62916667 0.5875     0.64583333 0.60833333 0.60416667
 0.60833333 0.6        0.59166667]
0.3333333333333333
(256, 15)
[0.0625     0.234375   0.57421875 1.         0.71875    0.71875
 0.59765625 0.62890625 0.60546875 0.66015625 0.609375   0.625
 0.62890625 0.625      0.57421875]
0.8
(279, 15)
[0.12544803 0.25448029 0.46236559 0.64157706 1.         0.7921147
 0.6702509  0.68100358 0.65232975 0.72759857 0.68458781 0.66308244
 0.67383513 0.67383513 0.61290323]
0.7333333333333333
(284, 15)
[0.1443662  0.22535211 0.3943662  0.538

In [134]:
print(X.iloc[2])
print(X_new[0,2])
print(yhat[0][:10])
print(yhat_new[0,0,:10])

Married                                    0
Single                                     1
Age_lt_25                                  1
Age_in_25_to_40                            0
Age_in_40_to_59                            0
Age_geq_60                                 0
EducationLevel                             3
MaxBillAmountOverLast6Months             580
MaxPaymentAmountOverLast6Months          100
MonthsWithZeroBalanceOverLast6Months       0
MonthsWithLowSpendingOverLast6Months       0
MonthsWithHighSpendingOverLast6Months      5
MostRecentBillAmount                     470
MostRecentPaymentAmount                  100
TotalOverdueCounts                         1
TotalMonthsOverdue                         8
HistoryOfOverduePayments                   1
Name: 16, dtype: int64
[  0   1   1   0   0   0   3 580 100   0   0   5 470 105   1   8   1]
[1. 1. 0. 1. 1. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [90]:
print(xs[:10])
for j_clf in range(len(good_classifiers)):
    print("after", preds[j_clf][:10])
    print("befor", yhat[j_clf][np.flatnonzero(yhat[0] <= 0)][:10])

[[   0    1    1    0    0    0    3  580  100    0    0    5  470  105
     1    8    1]
 [   0    1    0    1    0    0    2 1940  150    0    6    0 1780   80
     1    9    1]
 [   1    0    0    0    1    0    2   80   30    0    6    0   50   42
     1    9    1]
 [   0    1    0    1    0    0    3 5190  350    0    0    6 4900  230
     1    9    1]
 [   0    1    0    1    0    0    3  770   80    0    0    0  690    0
     1    8    1]
 [   0    1    0    1    1    0    2 5190  460    0    0    5 4450  460
     1   10    1]
 [   0    1    0    1    0    0    3 1550   60    0    0    6 1440   60
     1    7    1]
 [   0    1    0    1    0    0    2 5790  380    0    0    3 5790  270
     1   10    1]
 [   1    0    0    1    0    0    1 2910  150    0    0    0 2750  168
     1   10    1]
 [   1    0    0    1    0    0    1 4430  330    0    0    6 4160  200
     1    9    1]]
after [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
befor [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
after [0. 0. 0. 0. 1. 0

Run Recourse Audit for each classifier on Training Data (Takes some time!)

In [29]:
audit = {"cost": [], "feasible": []}
for j, clf in enumerate(good_classifiers):
    auditor = RecourseAuditor(action_sets[j], coefficients = clf.coef_[0], intercept = clf.intercept_[0])
    audit_df = auditor.audit(X)  ## matrix of features over which we will perform the audit.
    audit["feasible"].append(audit_df['feasible'].mean())
    audit["cost"].append(audit_df['cost'].mean())
audit

{'cost': [0.04378643277938842,
  0.048634724098296535,
  0.053305009106026625,
  0.05550058975401354,
  0.05345617409985927,
  0.05561050869289419,
  0.055975008326301985,
  0.056043761182863215,
  0.05742682141530978,
  0.0566258921611109,
  0.05879636740191718,
  0.05762381107305099,
  0.058091545384215626,
  0.05818023972797271,
  0.056392277783136664],
 'feasible': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0]}